### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
19,39399727,91794,79085,51530,0,0,999,5,2022-09-01 23:30,2022-09-01 23:30,5,0,1629,4,1
20,39398690,88725,79085,3,0,0,999,6,2022-09-02 21:59,2022-09-02 21:59,3,0,2125,0,1
21,39401984,13897,79085,51530,0,0,1000,0,2022-09-03 19:30,2022-09-03 19:30,0,0,3841,4,1
22,39236909,79085,79853,2076,21,57,1000,1,2022-09-04 21:59,2022-09-04 22:00,3,0,5097,3,1
23,39401978,79085,5245,51530,0,0,1000,2,2022-09-05 10:30,2022-09-05 10:30,1,1,2857,4,1
24,39403456,43862,79085,51530,0,0,1000,3,2022-09-06 19:30,2022-09-06 19:30,0,0,4583,3,1
25,39236895,6298,79085,2076,22,57,1000,4,2022-09-07 21:59,2022-09-07 22:00,2,4,10690,4,1
26,39408639,76607,79085,51530,0,0,1000,5,2022-09-08 19:30,2022-09-08 19:30,1,12,2048,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

229

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,91794,24049,34375,2,26,1,1,0,4-3-2 .02,9,11,10,39399727
1,79085,45924,35598,1,2,1,0,0,4-2-3-1a,7,6,8,39399727
0,88725,27749,37210,2,10,2,0,1,Scotti,9,4,7,39398690
1,79085,42286,32825,2,4,4,0,0,5-3-2a,5,5,6,39398690
0,13897,24337,40530,1,4,2,0,1,kiko,8,8,9,39401984
1,79085,45831,29638,1,2,3,0,0,4-2-3-1a,7,6,7,39401984
0,79085,24983,37301,1,16,4,1,0,4-2-3-1a,8,7,7,39236909
1,79853,45165,32847,0,3,3,0,0,442-Rafael,5,7,7,39236909
0,79085,46382,34762,0,1,2,0,0,5-4-1a,6,6,7,39401978
1,5245,23673,35293,3,11,3,1,0,Angrep-4-1-4-1,9,8,8,39401978


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

458

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2076

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,79085,22,62,20,2,0,126,4,62622126020011,2076
1,6298,22,52,17,1,4,71,12,52559071017012,2076
2,122540,22,50,16,2,4,78,6,50572078016010,2076
3,123911,22,43,13,4,5,59,15,43544059013007,2076
4,78473,22,43,14,1,7,66,25,43541066014004,2076
5,120765,22,36,11,3,8,38,25,36513038011008,2076
6,79853,22,31,10,1,11,64,37,31527064010009,2076
7,142717,22,23,7,2,13,26,73,23453026007002,2076
8,142636,22,19,6,1,15,16,62,19454016006006,2076
9,142663,22,16,5,1,16,18,76,16442018005003,2076


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

2

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()